# Data Extraction
## Purpose
To collect data from database into a local sqllite database. This avoids maintaining a constant connection to the online private database. 

## Imports
Import needed python modules

In [15]:
import sqlite3 # for backup data

import pandas as pd
from sqlalchemy import create_engine, text

#Retrieve connection sqlalchemy styled connection strings to source databases
# Exact connection parameters are proprietary and are stored locally.
from sqlconnection import connection_waterbox, connection_process
from raw_tensile_chemistry import get_sqis_tensile
from sizeconversion import frac_to_dec, frac_to_dec_str

# Global variables
# Set date start for retrieval
start_date = '2021-12-01'
local_db = 'raw_download.db'

## Retrieve the necessary Tables
Waterbox information is saved to the `WB` table. The output temperature is saved to the `LH` table. Categorical data is saved in a seperate a process database under a joining of `Coil` and `Schedule` tables. 

Using pandas read_sql, three tables are joined in the SQL statement to gather the process data. 
The waterbox (`WB`) table contains the core process variables. The laying head (`LH`) is the next production unit after the waterboxes. It contains the output variable, final temperature. It is retrieved using pandas read_sql. The formatting is improved by renaming columns and dropping system columns. The No Twist Mill (`PWB1_NTM`) is a rolling stand which precedes and feeds the waterboxes. This table contains some additional information, such as the product speed, and temperature entering the waterboxes. 

The `Coil_ID` is used as the index, and in the waterbox table has a trailing space, which is removed in the SQL select statement. 

In [2]:
sql = f''' --begin-sql
SELECT
    TRIM(WB.[Coil_ID]) AS Coil_ID
    , PWB1_NTM.[_TimeStamp] as Time
    , WB.[WB1_Select]
    , WB.[WB1_CoolValOpn]
    , WB.[WB1_WStripValOpn]
    , WB.[WB1_Zone1ValOpn]
    , WB.[WB1_Zone2ValOpn]
    , WB.[WB1_AirStripValOpn]
    --, WB.[WB1_TempControlSel]
    --, WB.[WB1_TempControlSP]
    , WB.[WB1_FlowControlSel]
    , WB.[WB1_FlowControlSP]
    , WB.[WB1_WaterFlowTr_GPM]
    , WB.[WB1_WaterFlowTr_SD]
    , WB.[WB1_WaterFlowRef_GPM]
    , WB.[WB1_WaterFlowRef_SD]
    , WB.[WB1_LinePressTr_PSI]
    , WB.[WB2_Select]
    , WB.[WB2_CoolValOpn]
    , WB.[WB2_WStripValOpn]
    , WB.[WB2_Zone1ValOpn]
    , WB.[WB2_Zone2ValOpn]
    , WB.[WB2_AirStripValOpn]
    --, WB.[WB2_TempControlSel]
    --, WB.[WB2_TempControlSP]
    , WB.[WB2_FlowControlSel]
    , WB.[WB2_FlowControlSP]
    , WB.[WB2_WaterFlowTr_GPM]
    , WB.[WB2_WaterFlowTr_SD]
    , WB.[WB2_WaterFlowRef_GPM]
    , WB.[WB2_WaterFlowRef_SD]
    , WB.[WB2_LinePressTr_PSI]
    , WB.[WB3_Select]
    , WB.[WB3_CoolValOpn]
    , WB.[WB3_WStripValOpn]
    , WB.[WB3_Zone1ValOpn]
    , WB.[WB3_Zone2ValOpn]
    , WB.[WB3_AirStripValOpn]
    --, WB.[WB3_TempControlSel]
    --, WB.[WB3_TempControlSP]
    , WB.[WB3_FlowControlSel]
    , WB.[WB3_FlowControlSP]
    , WB.[WB3_WaterFlowTr_GPM]
    , WB.[WB3_WaterFlowTr_SD]
    , WB.[WB3_WaterFlowRef_GPM]
    , WB.[WB3_WaterFlowRef_SD]
    , WB.[WB3_LinePressTr_PSI]
    --, WB.[AWB3_Temp]
    --, WB.[AWB3_SD]
    --, WB.[AWB3_TempRateOfChange]
    , WB.[WB4_Select]
    , WB.[WB4_CoolValOpn]
    , WB.[WB4_WStripValOpn]
    , WB.[WB4_Zone1ValOpn]
    , WB.[WB4_Zone2ValOpn]
    , WB.[WB4_Zone3ValOpn]
    , WB.[WB4_AirStripValOpn]
    --, WB.[WB4_TempControlSel]
    --, WB.[WB4_TempControlSP]
    , WB.[WB4_FlowControlSel]
    , WB.[WB4_FlowControlSP]
    , WB.[WB4_WaterFlowTr_GPM]
    , WB.[WB4_WaterFlowTr_SD]
    , WB.[WB4_WaterFlowRef_GPM]
    , WB.[WB4_WaterFlowRef_SD]
    , WB.[WB4_LinePressTr_PSI]
    --, WB.[AWB4_Temp]
    --, WB.[AWB4_SD]
    --, WB.[AWB4_TempRateOfChange]
    , WB.[WB5_Select]
    , WB.[WB5_CoolValOpn]
    , WB.[WB5_WStripValOpn]
    , WB.[WB5_Zone1ValOpn]
    , WB.[WB5_Zone2ValOpn]
    , WB.[WB5_Zone3ValOpn]
    , WB.[WB5_AirStripValOpn]
    --, WB.[WB5_TempControlSel]
    --, WB.[WB5_TempControlSP]
    , WB.[WB5_FlowControlSel]
    , WB.[WB5_FlowControlSP]
    , WB.[WB5_WaterFlowTr_GPM]
    , WB.[WB5_WaterFlowTr_SD]
    , WB.[WB5_WaterFlowRef_GPM]
    , WB.[WB5_WaterFlowRef_SD]
    , WB.[WB5_LinePress_PSI]
    , PWB1_NTM.[NTM_MatSpeed_MPS]
    , PWB1_NTM.[NTM_MatSpeed_SD]
    , PWB1_NTM.[NTM_WBValRef]
    , PWB1_NTM.[NTMST_TorquePercent]
    , PWB1_NTM.[NTMST_Torque_SD]
    , PWB1_NTM.[ANTM_Temp]
    , PWB1_NTM.[ANTM_SD] AS ANTM_Temp_SD
    , PWB1_NTM.[ANTM_TempRateOfChange] AS ANTM_Temp_RateOfChange
    , LH.[LH_Temp]
    , LH.[LH_SD] as LH_Temp_SD
    , LH.[LH_TempRateOfChange] AS LH_Temp_RateOfChange
FROM dbo.WB, dbo.LH, dbo.PWB1_NTM
WHERE
    (dbo.WB.COIL_ID != '') AND
    PWB1_NTM._TimeStamp >= '{start_date}' AND
    dbo.WB.COIL_ID = dbo.LH.COIL_ID AND
    dbo.WB.COIL_ID = dbo.PWB1_NTM.COIL_ID
order by PWB1_NTM._TimeStamp
-- end-sql
'''
connection_str = connection_waterbox()
engine = create_engine(connection_str)
connection = engine.connect()

waterbox_raw = pd.read_sql(
    text(sql), 
    connection, 
    index_col='Coil_ID'
)

connection.close()

waterbox_raw.head()

,Time,WB1_Select,WB1_CoolValOpn,WB1_WStripValOpn,WB1_Zone1ValOpn,WB1_Zone2ValOpn,WB1_AirStripValOpn,WB1_FlowControlSel,WB1_FlowControlSP,WB1_WaterFlowTr_GPM,...,NTM_MatSpeed_SD,NTM_WBValRef,NTMST_TorquePercent,NTMST_Torque_SD,ANTM_Temp,ANTM_Temp_SD,ANTM_Temp_RateOfChange,LH_Temp,LH_Temp_SD,LH_Temp_RateOfChange
Coil_ID,,,,,,,,,,,,,,,,,,,,,
6263890070,2021-12-01 03:46:03,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.048897,...,0.001064,100.0,17.838600,4.23196,1778.130005,9.53776,-0.168347,1758.589966,11.12110,-0.096117
6263890090,2021-12-01 03:57:07,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.048006,...,0.001064,100.0,17.740700,4.21151,1782.400024,13.21290,-0.368157,1761.780029,11.19780,-0.300954
6263890100,2021-12-01 03:58:39,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.047660,...,0.001064,100.0,17.862400,4.22400,1772.550049,11.37910,-0.230426,1754.020020,9.86034,-0.242252
6263890130,2021-12-01 04:03:47,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.046780,...,0.001064,100.0,17.778799,4.27374,1777.979980,11.75370,-0.305212,1756.199951,10.51100,-0.327176
6263890340,2021-12-01 04:31:37,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.043272,...,0.001064,100.0,17.659599,4.20141,1763.819946,8.64027,-0.129166,1757.500000,7.03784,-0.092242


In [3]:
waterbox_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130041 entries, 6263890070 to 6347860540
Data columns (total 79 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Time                    130041 non-null  datetime64[ns]
 1   WB1_Select              130041 non-null  float64       
 2   WB1_CoolValOpn          130041 non-null  float64       
 3   WB1_WStripValOpn        130041 non-null  float64       
 4   WB1_Zone1ValOpn         130041 non-null  float64       
 5   WB1_Zone2ValOpn         130041 non-null  float64       
 6   WB1_AirStripValOpn      130041 non-null  float64       
 7   WB1_FlowControlSel      130041 non-null  float64       
 8   WB1_FlowControlSP       130041 non-null  float64       
 9   WB1_WaterFlowTr_GPM     130041 non-null  float64       
 10  WB1_WaterFlowTr_SD      130041 non-null  float64       
 11  WB1_WaterFlowRef_GPM    130041 non-null  float64       
 12  WB1_WaterFlowRef_SD   

Changing the target database, the categorical process data is pulled from a joining of two separate tables. 

In [16]:
# Categorical Table
sql = f'''
    SELECT DISTINCT
        --CAST(
            CONCAT(
                Coil.[HeatNumber]
                , RIGHT('000' + CAST([Sequence] As varchar), 3)
                , [SplitSequence]
            --) AS BIGINT
        ) AS Coil_ID
        ,[Coil].[Grade] AS [Coil_Grade]
        ,[Coil].[CoilSize] AS [Coil_Size]
        ,[RolledTime] AS [Time_Rolled]
    FROM [Coil] 
        INNER JOIN [Schedule] ON [Coil].[ScheduleID] = [Schedule].[ID]
    WHERE RolledTime >= '{start_date}'
    order by Time_rolled
'''

connection_str = connection_process()
engine = create_engine(connection_str)
connection = engine.connect()

process_raw = pd.read_sql(
    text(sql), 
    connection,
    index_col='Coil_ID',
)

process_raw['Coil_Size_Dec'] = process_raw['Coil_Size'].map(frac_to_dec, na_action='ignore')
process_raw['Coil_Size_Frac'] = process_raw['Coil_Size']
process_raw['Coil_Size'] = process_raw['Coil_Size'].map(frac_to_dec_str, na_action='ignore')

connection.close()

process_raw.head()

,Coil_Grade,Coil_Size,Time_Rolled,Coil_Size_Dec,Coil_Size_Frac
Coil_ID,,,,,
6262930370,SCRAP,#6,2021-12-01 00:19:36,NaN,#6
6262930380,SCRAP,#6,2021-12-01 01:13:49,NaN,#6
6263890050,RB-15-37,#6,2021-12-01 03:10:33,NaN,#6
6263890060,RB-15-37,#6,2021-12-01 03:23:45,NaN,#6
6263890070,RB-15-37,#6,2021-12-01 03:43:22,NaN,#6


In [19]:
ten_chem_raw = get_sqis_tensile(start=start_date)
ten_chem_raw.head()

c:\Users\blakamp\Box\Documents\Regis\MSDS692-696 Practicum\raw_tensile_chemistry.py:554: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('\.','')


Tensile                                                       ... Chem  \
  TestNum Coil DimIni DimRed  YS      UT  EL     RA     OV  WT  ...    H   
0     1.0  1.0  0.562  0.434 NaN  113.98 NaN  40.29  0.006 NaN  ...  0.0   
1     2.0  2.0  0.562  0.424 NaN  112.25 NaN  43.11  0.006 NaN  ...  0.0   
2     3.0  3.0  0.562  0.432 NaN  113.06 NaN  40.98  0.006 NaN  ...  0.0   
3     4.0  4.0  0.562  0.431 NaN  114.28 NaN  41.10  0.006 NaN  ...  0.0   
4     5.0  5.0  0.562  0.420 NaN  115.19 NaN  44.20  0.006 NaN  ...  0.0   

         MTR                                                           Coil  \
        Date     Status        CreationDate             Updated          ID   
0 2021-12-01  OK-TESTED 2021-12-01 13:49:51 2021-12-01 13:49:51  6266060010   
1 2021-12-01  OK-TESTED 2021-12-01 13:49:51 2021-12-01 13:49:51  6266060020   
2 2021-12-01  OK-TESTED 2021-12-01 13:49:51 2021-12-01 13:49:51  6266060030   
3 2021-12-01  OK-TESTED 2021-12-01 13:49:51 2021-12-01 13:49:51  6266060040   
4 2021-12-01  OK-TESTED 2021-12-01 13:49:51 2021-12-01 13:49:51  6266060050   

  Schedule               Tensile      Chem  
  Size_Dec Size_Frac         PTS        Di  
0   0.5625      9/16  107.055186  1.064379  
1   0.5625      9/16  107.055186  1.064379  
2   0.5625      9/16  107.055186  1.064379  
3   0.5625      9/16  107.055186  1.064379  
4   0.5625      9/16  107.055186  1.064379  

[5 rows x 55 columns]

## Backup Tables
Using sqlite3 library, copies of each table is saved for offline retrieval, and uploading for share. 

In [21]:
con = sqlite3.connect(local_db)
wb_rows = waterbox_raw.to_sql('Waterbox', con, if_exists='replace')
proc_rows = process_raw.to_sql('Process', con, if_exists='replace')

# convert multiindex to flat before upload
ten_chem_raw.columns = [
    f'{x}_{y}' for x, y in 
    ten_chem_raw.columns.to_list()
]
ten_rows = ten_chem_raw.to_sql('Ten_Chem', con, if_exists='replace')
f'Records uploaded: Waterbox {wb_rows}, Process {proc_rows}, Tensile/Chemistry {ten_rows}'


19819

## Retrieve Backups if Necessary

In [22]:
con = sqlite3.connect(local_db)
# waterbox data first
sql = 'Select * FROM Waterbox'
waterbox_raw = pd.read_sql(sql, con, parse_dates='Time').set_index('Coil_ID')
sql = 'Select * FROM Process'
process_raw = pd.read_sql(sql, con, parse_dates='Time_Rolled').set_index('Coil_ID')
sql = 'Select * FROM Ten_Chem'
ten_chem_raw = pd.read_sql(sql, con, parse_dates='MTR_Date').set_index('Coil_ID')

# Correct multiindex for `ten_chem_raw`
#ten_chem_raw.columns = pd.MultiIndex.from_tuples(
#    ten_chem_raw.columns.str.split('_',1).tolist()
#)



# Exploration of Data

## Merge Tables
The waterbox data will be left joined to categorical data, using the coil_ID. The collection method of the waterbox can lead to some data loss. The process data is a more accurate tracking method. First a comparison of the two sizes of the data frames indicates about 1.4% data loss.

In [23]:
print(f'Process frame size {process_raw.shape}.',
    f'Waterbox frame size {waterbox_raw.shape}.',
    f'Unmatchable records {(process_raw.shape[0]-waterbox_raw.shape[0])/process_raw.shape[0]*100:0.2}%')

Process frame size (131644, 5). Waterbox frame size (130041, 79). Unmatchable records 1.2%


Now to join the tables. Right joining the process data to waterbox data places the process data first. After joining a determination of the number of coils which are unmatched are evaluated. 

In [24]:
merged_raw = pd.merge(process_raw, waterbox_raw, how='right', left_index=True, right_index=True)
print('Shape of merged frame:', merged_raw.shape)
merged_raw.head()

Shape of merged frame: (130041, 84)


,Coil_Grade,Coil_Size,Time_Rolled,Coil_Size_Dec,Coil_Size_Frac,Time,WB1_Select,WB1_CoolValOpn,WB1_WStripValOpn,WB1_Zone1ValOpn,...,NTM_MatSpeed_SD,NTM_WBValRef,NTMST_TorquePercent,NTMST_Torque_SD,ANTM_Temp,ANTM_Temp_SD,ANTM_Temp_RateOfChange,LH_Temp,LH_Temp_SD,LH_Temp_RateOfChange
Coil_ID,,,,,,,,,,,,,,,,,,,,,
1137580520,PA28135,#6,2021-12-01 08:41:33,NaN,#6,2021-12-01 08:44:13,0.0,0.0,0.0,1.0,...,0.001064,100.000000,18.193501,4.36095,1758.010010,13.7249,0.569037,1751.430054,11.3097,0.553520
1137580550,PA28135,#6,2021-12-01 08:45:04,NaN,#6,2021-12-01 08:47:43,0.0,0.0,0.0,1.0,...,0.001064,100.000000,18.306700,4.36990,1753.099976,15.0344,0.709423,1747.040039,12.9610,0.631423
1137940520,PC1360,0.562,2022-02-21 18:18:02,0.5625,"9/16""",2022-02-21 18:20:46,0.0,0.0,0.0,1.0,...,0.005203,42.194698,23.707800,5.99103,1772.630005,12.1242,-0.329673,1623.839966,18.4505,-0.441458
1137940530,PC1360,0.562,2022-02-21 18:19:27,0.5625,"9/16""",2022-02-21 18:22:10,0.0,0.0,0.0,1.0,...,0.005203,42.227699,23.537100,5.88219,1776.420044,10.2621,-0.368416,1628.859985,19.5825,-0.450046
1138061050,PC1360,0.250,2021-12-14 18:10:43,0.2500,"1/4""",2021-12-14 18:14:01,0.0,0.0,0.0,1.0,...,0.013417,100.000000,44.433201,12.91770,1789.239990,11.4778,-0.018902,1852.130005,28.8514,-26.502199


## Characteristic Review
To understand the impact of cooling, having a starting and finishing temperature will be important. The ANTM_TEMP (after NTM temperature) is missing almost 1400 values, but is only a minor loss compared to the 130000 initial records. 

In [26]:
merged_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130041 entries, 1137580520 to 6347900640
Data columns (total 84 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Coil_Grade              130041 non-null  object        
 1   Coil_Size               130041 non-null  object        
 2   Time_Rolled             130041 non-null  datetime64[ns]
 3   Coil_Size_Dec           108586 non-null  float64       
 4   Coil_Size_Frac          130041 non-null  object        
 5   Time                    130041 non-null  datetime64[ns]
 6   WB1_Select              130041 non-null  float64       
 7   WB1_CoolValOpn          130041 non-null  float64       
 8   WB1_WStripValOpn        130041 non-null  float64       
 9   WB1_Zone1ValOpn         130041 non-null  float64       
 10  WB1_Zone2ValOpn         130041 non-null  float64       
 11  WB1_AirStripValOpn      130041 non-null  float64       
 12  WB1_FlowControlSel    

In [25]:
merged_raw.isna().sum()[merged_raw.isna().sum() > 0]

Coil_Size_Dec             21455
NTM_MatSpeed_MPS             12
NTM_MatSpeed_SD              12
NTM_WBValRef                 12
NTMST_TorquePercent          12
NTMST_Torque_SD              12
ANTM_Temp                  1363
ANTM_Temp_SD               1349
ANTM_Temp_RateOfChange     1363
LH_Temp                      93
LH_Temp_SD                   79
LH_Temp_RateOfChange         93
dtype: int64